In [77]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import FloatType
import datetime

In [78]:
# language detection dependentcies
from lingua import Language, LanguageDetectorBuilder
import re

In [79]:
spark = SparkSession.builder.appName("Preprocessing").getOrCreate()
sc = spark.sparkContext

In [80]:
# load the dataset
df = spark.read.csv("subset.csv", header=True, inferSchema=True)

In [81]:
# for now we'll do the language detection first, but this won't work at scale so we'll have to do somethign else later
languages = [Language.ENGLISH, Language.SPANISH]
pattern = r"\s*[\(\[].*?[\)\]]"

def detect_language_udf(title, album):
    cleaned_title = re.sub(pattern, "", title, flags=re.IGNORECASE).strip()
    cleaned_album = re.sub(pattern, "", album, flags=re.IGNORECASE).strip()
    combined_text = f"{cleaned_title} {cleaned_album}"
    if not combined_text.strip():
        return 0.0  # return 0.0 for empty strings to just guess - we might want to make this more sophisticated later

    detector = LanguageDetectorBuilder.from_languages(*languages).build()
    language = detector.detect_language_of(combined_text)
    return float(languages.index(language))

detect_language = F.udf(detect_language_udf, FloatType())

df_with_lang = df.withColumn("language_id", detect_language(F.col("title"), F.col("album")))

new_df = df_with_lang.select("title", "album", "language_id")
new_df.show(5)

+--------------------+--------------------+-----------+
|               title|               album|language_id|
+--------------------+--------------------+-----------+
|Chantaje (feat. M...|           El Dorado|        1.0|
|Vente Pa' Ca (fea...|Vente Pa' Ca (fea...|        1.0|
|Reggaetón Lento (...|        Primera Cita|        1.0|
|              Safari|             Energía|        1.0|
|         Shaky Shaky|         Shaky Shaky|        0.0|
+--------------------+--------------------+-----------+
only showing top 5 rows


In [82]:
# remove the following columns: urls, track_id, data, available markets, id, and date
# also remove region, and name because they are strings not worth embedding for now
# also remove chart because I don't think there are enough charts for this to be relevant
# finally, remove index because dataframes already have an index
columns_to_remove = [
    "urls", "track_id", "data", "available_markets", "id", "url", "date",
    "region", "name", "chart",
    "rank", "streams", "trend", "popularity", "duration_ms", "release_date", "af_time_signature", "af_key", "af_mode", "af_liveness"
]
df = df.drop(*columns_to_remove)

# df = df.drop(*columns_to_remove)
df.show(5)

+----------+--------------------+--------------------+--------------------+--------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+
|Unnamed: 0|               title|              artist|               album|explicit|af_danceability|af_energy|af_loudness|af_speechiness|af_acousticness|af_instrumentalness|af_valence|af_tempo|
+----------+--------------------+--------------------+--------------------+--------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+
|         0|Chantaje (feat. M...|             Shakira|           El Dorado|   false|          0.852|    0.773|     -2.921|        0.0776|          0.187|            3.05E-5|     0.907| 102.034|
|         1|Vente Pa' Ca (fea...|        Ricky Martin|Vente Pa' Ca (fea...|   false|          0.663|     0.92|      -4.07|         0.226|        0.00431|            1.69E-5|     0.533|  99.935|
|         2|Reggaetón Lento (.

In [83]:
# remove duplicates by checking to see if any titles match
df = df.dropDuplicates(["title"])

In [84]:
# convert the date columns into a float representing the year
def date_to_year(date):
    try:
        return float(date.year)
    except:
        return None
    
date_to_year_udf = F.udf(date_to_year, FloatType())
# df = df.withColumn("date", date_to_year_udf(F.col("release_date")))
# df = df.drop("release_date")
df.show(5)

+----------+--------------------+-----------------+--------------------+--------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+
|Unnamed: 0|               title|           artist|               album|explicit|af_danceability|af_energy|af_loudness|af_speechiness|af_acousticness|af_instrumentalness|af_valence|af_tempo|
+----------+--------------------+-----------------+--------------------+--------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+
|        34|"CAN'T STOP THE F...|Justin Timberlake|"CAN'T STOP THE F...|   false|          0.666|     0.83|     -5.715|        0.0751|         0.0123|                0.0|     0.702|  113.03|
|        33|           24K Magic|       Bruno Mars|           24K Magic|   false|          0.818|    0.803|     -4.282|        0.0797|          0.034|                0.0|     0.632|  106.97|
|       114|                 743|         Mir

In [85]:
# convert the trend column into a scale from 0-2
# def trend_to_scale(trend):
#     if trend == "MOVE_UP":
#         return 2.0
#     elif trend == "MOVE_DOWN":
#         return 0.0
#     else:
#         return 1.0

# trend_to_scale_udf = F.udf(trend_to_scale, FloatType())
# df = df.withColumn("trend", trend_to_scale_udf(F.col("trend")))
# df.show(5)

In [86]:
# remove explicit content, and then remove the explicit column
df = df.filter(F.col("explicit") == False)
df = df.drop("explicit")
df.show(5)

+----------+--------------------+-----------------+--------------------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+
|Unnamed: 0|               title|           artist|               album|af_danceability|af_energy|af_loudness|af_speechiness|af_acousticness|af_instrumentalness|af_valence|af_tempo|
+----------+--------------------+-----------------+--------------------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+
|        34|"CAN'T STOP THE F...|Justin Timberlake|"CAN'T STOP THE F...|          0.666|     0.83|     -5.715|        0.0751|         0.0123|                0.0|     0.702|  113.03|
|        33|           24K Magic|       Bruno Mars|           24K Magic|          0.818|    0.803|     -4.282|        0.0797|          0.034|                0.0|     0.632|  106.97|
|       114|                 743|         Miranda!|              Fuerte|          0.849|  

In [87]:
# convert the rank column to a float
# df = df.withColumn("rank", F.col("rank").cast(FloatType()))
# df.show(5)

In [88]:
df = df.drop("Unnamed: 0")
df.show(5)

+--------------------+-----------------+--------------------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+
|               title|           artist|               album|af_danceability|af_energy|af_loudness|af_speechiness|af_acousticness|af_instrumentalness|af_valence|af_tempo|
+--------------------+-----------------+--------------------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+
|"CAN'T STOP THE F...|Justin Timberlake|"CAN'T STOP THE F...|          0.666|     0.83|     -5.715|        0.0751|         0.0123|                0.0|     0.702|  113.03|
|           24K Magic|       Bruno Mars|           24K Magic|          0.818|    0.803|     -4.282|        0.0797|          0.034|                0.0|     0.632|  106.97|
|                 743|         Miranda!|              Fuerte|          0.849|    0.759|     -6.232|        0.0304|         0.0951|            1.2

In [89]:
df = df.withColumn("language_id", detect_language(F.col("title"), F.col("album")))
df.show(5)

+--------------------+-----------------+--------------------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+-----------+
|               title|           artist|               album|af_danceability|af_energy|af_loudness|af_speechiness|af_acousticness|af_instrumentalness|af_valence|af_tempo|language_id|
+--------------------+-----------------+--------------------+---------------+---------+-----------+--------------+---------------+-------------------+----------+--------+-----------+
|"CAN'T STOP THE F...|Justin Timberlake|"CAN'T STOP THE F...|          0.666|     0.83|     -5.715|        0.0751|         0.0123|                0.0|     0.702|  113.03|        0.0|
|           24K Magic|       Bruno Mars|           24K Magic|          0.818|    0.803|     -4.282|        0.0797|          0.034|                0.0|     0.632|  106.97|        0.0|
|                 743|         Miranda!|              Fuerte|          0.849|    0.75

In [90]:
# final step: convert it to be a column like this: [title, arist, features]
# df = df.withColumn("features", F.array([F.col(c).cast(FloatType()) for c in df.columns if c not in ["title", "artist"]]))
# df = df.select("title", "artist", "features")
# df.show(5)

In [91]:
# df = df.withColumn(
#     "features",
#     F.concat(
#         "features",
#         F.array(detect_language(F.col("title"), F.col("artist")))   # appending language id calculation
#     )
# )
# df.show(5)

In [92]:
df.write.csv("numpy_prepped.csv", header=True, mode="overwrite")